In [1]:
import os
import datetime
import functools
from lxml import etree as et

In [2]:
from helpers import helpers
from helpers import boe
from helpers import boe_diary_processing
from helpers import boe_diary_entry_processing

In [3]:
date = datetime.datetime(2020, 5, 7)
base_dir = '../src/tests/mockup_data'

# BOE diary summary processing

    1. Retrieval, parse and entry extraction
    2. Entry processing
    3. Basic stats

## 1. Retrieval, parse and entry extraction

In [4]:
def date_2_output_path(date: datetime.datetime, base_dir: str) -> str:
    '''Returns the local filesystem output location for the diary of the given date.'''

    if not isinstance(base_dir, str): 
        raise TypeError(f'Expected a str and got a {type(base_dir)} for base_dir.')

    if not isinstance(date, datetime.datetime):
        raise TypeError(f'Expected datetime.datetime and got {type(date)}')

    return os.path.join(base_dir,
                     'diaries',
                     f"boe_diary_raw_{date.strftime('%d_%m_%Y')}.xml")
                        
def entry_id_2_output_path (entry_id: str, base_dir: str) -> str:
    '''Create local filesystem path for the diary entry output.'''

    if not isinstance(base_dir, str): 
        raise TypeError(f'Expected a str and got a {type(base_dir)} for base_dir.')

    if not boe.is_valid_diary_entry_id(entry_id):
        raise ValueError(f"'{entry_id}' is not a valid entry id.")

    return os.path.join(base_dir,
                     'diary_entries', 
                     f"boe_diary_entry_raw_{entry_id}.xml")

In [5]:
with open(date_2_output_path(date, base_dir), 'r') as f:
    diary = f.read()
    
tree = et.fromstring(diary.encode())
entries = list(helpers.pipe(tree,
    boe_diary_processing.get_sections,
    boe_diary_processing.get_departments_per_section,
    boe_diary_processing.get_items_per_department,
    boe_diary_processing.get_details_per_item,
    functools.partial(map, lambda item: {**item, 'date':date.isoformat()})))
len(entries)

60

## 2. Entry processing

In [6]:
entry = entries[0]
entry

{'id': 'BOE-A-2020-4859',
 'epigraph': 'Ley Concursal',
 'section': '1',
 'department': 'MINISTERIO DE LA PRESIDENCIA, RELACIONES CON LAS CORTES Y MEMORIA DEMOCRÁTICA',
 'title': 'Real Decreto Legislativo 1/2020, de 5 de mayo, por el que se aprueba el texto refundido de la Ley Concursal.',
 'pdf_url': '/boe/dias/2020/05/07/pdfs/BOE-A-2020-4859.pdf',
 'xml_url': '/diario_boe/xml.php?id=BOE-A-2020-4859',
 'htm_url': '/diario_boe/txt.php?id=BOE-A-2020-4859',
 'date': '2020-05-07T00:00:00'}

In [7]:
with open(entry_id_2_output_path(entry['id'], base_dir), 'r') as f:
    entry_content = f.read()
    
tree = et.fromstring(entry_content.encode())
labels = boe_diary_entry_processing.get_labels_from_tree(tree)
references = boe_diary_entry_processing.get_references_from_tree(tree)
entry_type = boe_diary_entry_processing.get_type_from_tree(tree)

**BOE diary entry labels**

In [15]:
len(labels)

44

In [14]:
', '.join(labels)

'Cooperativas, Mercado de Valores, Herencias, Concurso de Acreedores, Estatuto de los Trabajadores, Entidades de crédito, Consorcio de Compensación de Seguros, Seguros, Agrupaciones de interés económico, Hipoteca, Pagos, Impuesto sobre Transmisiones Patrimoniales y Actos Jurídicos Documentados, Contratos de las Administraciones Públicas, Juzgados de lo Mercantil, Código Civil, Administración concursal, Justicia Gratuita, Sistema tributario, Créditos, Derecho Civil, Consumidores y usuarios, Derecho Mercantil, Trabajo y empleo, Seguridad Social, Sistema financiero, Enjuiciamiento Civil, Procedimiento Laboral, Suspensión de pagos, Oficina judicial, Sociedades Anónimas, Derecho Administrativo, Impuesto sobre el Valor Añadido, Procedimiento concursal, Sociedades, Transportes y tráfico, Quiebra, Administración de Justicia, Sociedades de Garantía Recíproca, Sociedades de Responsabilidad Limitada, Navegación aérea, Fundaciones, Arbitraje, Contratos, Código de Comercio'

**BOE diary entry references**

In [9]:
references

({'referenced': 'BOE-A-2015-10566',
  'type': 'DEROGA',
  'type_code': '210',
  'text': 'la disposición final 5 de la Ley 40/2015, de 1 de octubre',
  'category': 'previous'},
 {'referenced': 'BOE-A-2015-8469',
  'type': 'DEROGA',
  'type_code': '210',
  'text': 'el art. 1 y la disposición transitoria 1 de la Ley 25/2015, de 28 de julio',
  'category': 'previous'},
 {'referenced': 'BOE-A-2015-7897',
  'type': 'DEROGA',
  'type_code': '210',
  'text': 'la disposición final 5 de la Ley 20/2015, de 14 de julio',
  'category': 'previous'},
 {'referenced': 'BOE-A-2015-6789',
  'type': 'DEROGA',
  'type_code': '210',
  'text': 'la disposición final 5 de la Ley 11/2015, de 18 de junio',
  'category': 'previous'},
 {'referenced': 'BOE-A-2015-5744',
  'type': 'DEROGA',
  'type_code': '210',
  'text': 'la disposición final 5 de la Ley 9/2015, de 25 de mayo',
  'category': 'previous'},
 {'referenced': 'BOE-A-2015-4607',
  'type': 'DEROGA',
  'type_code': '210',
  'text': 'la disposición final 1 d

**Entry type**

In [10]:
entry_type

('Real Decreto Legislativo', '1310')